### Задача. Построить модель исходя из эмперических данных об иерирахическом делении цветков для определенного вида. 

Построим для начала модель для числа цветков центральной ветви.

In [48]:
central_stem <- c(468, 196, 82, 42, 18)

In [49]:
left_branch <- c(186, 138, 57, 20, 11)
right_branch <- c(161, 134, 57, 20, 13)

In [50]:
left_branch/central_stem

[1] 0.3974359 0.7040816 0.6951220 0.4761905 0.6111111

In [51]:
right_branch / central_stem

[1] 0.3440171 0.6836735 0.6951220 0.4761905 0.7222222

Из отношений количества цветков условной правой и левой ветвей на каждом этапе деления видно, что закономерность
такого соотношения имеется. При первом делении, где-то 40 % от числа цветков на центральной ветви уходит на правую и левую
ветви. 

Оценим, сколько цветков на каждом шаге сохраняется для центральной ветви.

In [52]:
stem_strength <- central_stem / (right_branch + left_branch + central_stem)
print(stem_strength)

[1] 0.5742331 0.4188034 0.4183673 0.5121951 0.4285714


In [53]:
summary(stem_strength)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.4184  0.4188  0.4286  0.4704  0.5122  0.5742 

Описательная статистика (хоть и для такой маленькой выборки), показывает, что в целом данные достаточно хорошо
сконцентрированы около значения 0.45 (т.к. median = 0.42, mean=0.47). Если говорить строго, то это значение, точнее
его теоретическое значение в модели, которую еще предстоит построить, необходимо будет оценивать исходя из
какого-либо подхода, например, методом максимального правдоподобия или методом наименьших квадратов.

## Построение модели роста центрального стержня

Учитывая, что относительное число цветков на центральной ветке сохраняется, можно предложить следующую модель 

$$s_n = \alpha\cdot s_{n-1} + \nu$$,

где $s_n$ -- количество цветков на центральной ветви на n-ом шаге, $\nu$ -- случайная величина с исзвестным распределением, например, распределением Пуассона (параметр распределения в этом случае нужно будет находить из эмпирических данных.

Оценка при условии, что $\nu$ будет иметь минимальную дисперсию будет иметь вид (эту формулу для данной модели можно вывести, но думаю, можно и найти где-нибудь в книге по теории случайных (марковских) процессов)
$$
\alpha = \frac{\sum x_i \cdot y_i}{x_i^2}
$$

In [54]:
x <- c(486, 196, 82, 42)
y<- c(196, 82, 42, 18)

In [55]:
alpha <- sum(x * y) / sum(x * x)
print(alpha)

[1] 0.4080819


In [56]:
sigma2_nu <- var(y - alpha * x)

In [57]:
mean_nu <- mean(y - alpha * x)

In [58]:
print(mean_nu); print(sigma2_nu)

[1] 2.271487
[1] 20.82329


**Из рассчетов следует:**

1. В среднем на каждом последующем шаге на центральной ветви остается 41% цветков от предыдущего шага ($\alpha=0.408$)
2. Погрешность модели, характеризуемая параметром $\nu$, достаточно мала: в среднем $\nu=2.27$, что свидетельствует об относительной точности модели, т.к. $2.27/486 << 1$. Дисперсия параметра $\nu$ (точнее ее оценка), также не очень велика в сравнении с абсолютными значениями количества цветков, т.е. $\sqrt{20.82}/486<<1$. Эти факты свидетельствуют в пользу предложенной модели.

### Исследование деления ветвей

#### Левая ветвь

In [59]:
left_branch_center <- c(186, 55, 16)
left_branch_shifted <- c(55, 16, 6)

In [60]:
left_branch_shifted/left_branch_center

[1] 0.2956989 0.2909091 0.3750000

In [36]:
alpha_left_branch <- sum(left_branch_shifted * left_branch_center) / sum(left_branch_center * left_branch_center)

In [37]:
print(alpha_left_branch)

[1] 0.2958524


Аналогично могут быть получены значение $\nu$ для левой ветви, а также оценки дисперсии. 

#### Правая ветвь

In [42]:
right_branch_center <- c(161, 47, 16)
right_branch_shifted <- c(47, 16, 6)

In [43]:
alpha_right_branch <- sum(right_branch_shifted * right_branch_center) / sum(right_branch_center * right_branch_center)

In [44]:
print(alpha_right_branch)

[1] 0.296449


Значения $\alpha$, полученные для левой и правой ветвей практически не отличаются друг от друга. Это свидетельствует в пользу выбранной модели.

Аналогично расчетам, выполненным для центральной ветви, могут быть получены значения дисперсии для $\nu$ правой ветви. 
Однако, количество измерений всего 3. Это может сказаться на точности оценки дисперсии.

#### Верхния ветвь (предварительный расчет)

In [46]:
x <- c(138,40,14)
y <- c(40,14,5)
alpha<-sum(x*y)/sum(x*x)
print(alpha)

[1] 0.2951056


#### Нижняя ветвь (предварительный расчет)

In [47]:
x <- c(134,41,15)
y <- c(41,15,5)
alpha<-sum(x*y)/sum(x*x)
print(alpha)

[1] 0.3113483


**Вывод:**

* Для всех ветвей оценка в смысле минимума дисперсии (по $\nu$) практически одинакова (т.е. параметр $\alpha$ почти один и тот же). Это говорит об адекватности модели.
* Построенную модель с найденными значениями параметров $\alpha$  и $\nu$ можно использовать для генерации виртуальных соцветий.
* Если образцы получены в разные годы, или произрастали в различных услових, то вполняя аналогичные оценки для двух или более условий среды и сравнивая полученные коэффииенты $\alpha_i$, $\nu_i$, где $i$ -- индекс, характеризующий условия среды, можно делать выводы о влиянии среды на структуру соцветия. Более того, можно говорить о том, в чем именно проявляется это влияние, например, по значениям $\alpha$ можно судить более развита центральная часть у соцветия или периферия.
* Рассмотренная модель далеко не единственная. Может быть предложена модель, например, следующего вида
$$
\frac{s_n}{s_{n-1}} = \alpha + \nu,
$$ где $\alpha=const$, а $\nu$ -- случайная величина. В этом случае, оценка  минимальной дисперсии будет иметь другой вид. И возникнет необходимость сравнения моделей, какая из них лучше описывает данные. Здесь, вполе возможно, будет уместно применение каких-либо критериев, типа aic, или оценка приемлемости моделей исходя из квадратов невязок.

Надеюсь эти рассуждения будут служить отправной точкой для проведения более детального научного исследования.  В целом, здесь уже прослеживается закономерность, хотя бы в том, что полученные оценки $\alpha$ мало отличаются друг от друга. 